Imports

In [ ]:
import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
import torch
from typing import Callable

import nn_util
# from nn_util import dist_and_proximity_loss, simple_dist_loss
from main import determine_device

device: torch.device = determine_device(1)

Heatmap generation functions

In [ ]:
def get_loss(loss_func:Callable, e:tuple[float,float], c:tuple[float,float], C:list[tuple[float,float]]) -> float:
    predicted_embedding = torch.Tensor([e]).to(device)
    target_labels: list[int] = [C.index(c)]
    class_embeddings = torch.Tensor(C).to(device)

    return loss_func(predicted_embedding, class_embeddings, target_labels, device)[0].item()

# 𝑒 = The embedding outputted by the neural network
# 𝑐  = The class embedding for the class of 𝑒
# 𝐶  = The set of class embeddings
def make_heatmap(
        x_min:int, x_max:int, y_min:int, y_max:int, resolution_upscale:float,
        loss_func:Callable, target:tuple[int,int], all_classes:list[tuple[int,int]]
        ) -> tuple[np.ndarray, tuple[int,int,int,int]]:
    
    heatmap = np.ndarray(shape=(
        int(np.floor((x_max-x_min)*resolution_upscale)),
        int(np.floor((y_max-y_min)*resolution_upscale)))
    )

    for x_i, y_i in np.ndindex(heatmap.shape):
        x = x_i / resolution_upscale + x_min
        y = y_i / resolution_upscale + y_min
        heatmap[x_i, y_i] = get_loss(loss_func, target, (x,y), [(x,y)] + all_classes, )

    return heatmap, (x_min, x_max, y_min, y_max)

Configuration

In [ ]:
x_min = -30
x_max = 20
y_min = -30
y_max = 20

resolution_upscale = 0.5

e = (-7,-3)
all_classes = [(-6, 3),(-1,1),(4,7),(6,0),(4,-6)]

def standardized_heatmap(loss_func): 
    return make_heatmap(x_min, x_max, y_min, y_max, resolution_upscale,
                              loss_func, e, all_classes)

def loss_heatmap(loss_func, title=None): 
    heatmap, edges = standardized_heatmap(loss_func)
    

    if title:
        plt.title(title)
        
    plt.imshow(heatmap.T, extent=edges, origin='lower')
    plt.scatter([-7],[-3], marker="$e$")

    cx = [c[0] for c in all_classes]
    cy = [c[1] for c in all_classes]
    plt.scatter(cx,cy, marker="$C$")
    plt.show()
    # plt.savefig("loss_plots/" + title)

Plotting

In [ ]:
loss_heatmap(nn_util.simple_dist_loss, title="Simple Loss")

In [ ]:
r = 1000
loss_heatmap(nn_util.dist_and_proximity_loss(r), title="Proximity Loss")

In [ ]:
loss_heatmap(nn_util.cone_loss_hyperparam(), title="Cone Loss")

In [ ]:
loss_heatmap(nn_util.comparison_dist_loss, title="Comparison Loss")

In [ ]:
loss_heatmap(nn_util.cosine_loss, title="Cosine loss")